In [ ]:
# ===== CONSTANTS =====
cache_file = "cache.csv"

# ===== CORE FUNCTIONS =====

# Liest eine CSV-Datei ein und trennt sie in Header (Spaltennamen) und Zeilen auf
def get_content(file):
    with open(file, "r") as data:
        lines = data.readlines()
        header = lines[0].strip().split(",")
    return header, lines

# Erzeugt oder überschreibt die Cache-Datei mit gegebenem Header und Zeilen
def generate_cache_file(header, lines, file=cache_file):
    with open(file, "w") as new_file:
        new_file.write(",".join(header) + "\n")
        for line in lines:
            if isinstance(line, list):
                new_file.write(",".join(line) + "\n")
            else:
                new_file.write(line + "\n")

# Gibt eine formatierte Tabelle auf der Konsole aus
def print_formatted_table(file=cache_file):
    header, lines = get_content(file)
    print("\nFormatted Table, from file:", file)
    print("With a total of " + str(len(lines) - 1) + " entrys.")
    print("-" * round(23.2 * len(header)))
    print(("| {:<20} " * len(header) + "|").format(*header))
    print("-" * round(23.2 * len(header)))
    for line in lines[1:]:
        print(("| {:<20} " * len(header) + "|").format(*line.strip().split(",")))

# Filtert Zeilen nach einem bestimmten Spaltenwert und schreibt das Ergebnis in die Cache-Datei
def get_rows_by_value(column, value, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print("Spalte '" + column + "' nicht gefunden.")
        return
    col_index = header.index(column)
    rows = []
    for line in lines[1:]:
        row = line.strip().split(",")
        if row[col_index] in value:
            rows.append(row)
    generate_cache_file(header, rows)
    print("\nOnly returned rows with value in column:", column)
    print("For Values:", value)
    if return_table:
        print_formatted_table()

# Sortiert die Tabelle nach einer bestimmten Spalte mithilfe des Bubble-Sorts
def sort_by(column, area, reverse=False, file=cache_file, return_table=False):
    header, lines = get_content(file)
    if column not in header:
        print("Spalte '" + column + "' nicht gefunden.")
        return
    if not area or area[1] == -1:
        area = [area[0], len(lines)]
    rows = [line.strip().split(",") for line in lines[area[0]:area[1]]]
    col_index = header.index(column)
    for i in range(len(rows)):
        for j in range(0, len(rows) - i - 1):
            try:
                a = int(rows[j][col_index])
                b = int(rows[j + 1][col_index])
            except ValueError:
                a = rows[j][col_index]
                b = rows[j + 1][col_index]
            if (a > b and not reverse) or (a < b and reverse):
                rows[j], rows[j + 1] = rows[j + 1], rows[j]
    generate_cache_file(header, rows)
    print("\nSorted by column:", column)
    print("Range:", area[0], "to", area[1])
    if return_table:
        print_formatted_table()

# Zählt die Gesamtanzahl der 'Menschen' in der Tabelle
def count_humans(file=cache_file):
    header, lines = get_content(file)
    if "Number" not in header:
        print("Spalte 'Number' nicht gefunden.")
        return 0
    col_index = header.index("Number")
    total_humans = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        try:
            total_humans += int(row[col_index])
        except ValueError:
            continue
    return total_humans

# Berechnet das Durchschnittsalter der 'Menschen' in der Tabelle
def average_age(file=cache_file):
    header, lines = get_content(file)
    if "YearOfBirth" not in header:
        print("Spalte 'YearOfBirth' nicht gefunden.")
        return None
    col_index = header.index("YearOfBirth")
    current_year = 2025
    total_age = 0
    count = 0
    for line in lines[1:]:
        row = line.strip().split(",")
        try:
            age = current_year - int(row[col_index])
        except ValueError:
            continue
        total_age += age
        count += 1
    if count == 0:
        print("No data available to calculate average age.")
        return None
    return total_age / count
